In [991]:
import spacy
import re

In [2]:
nlp = spacy.load("en_core_web_sm")

In [1011]:
unc_list0 = [] #you may specify your own list of uncountable nouns here 
unc_list = unc_list0 + ['experience', 'discussion', 'effort', 'support', 'stress', 'care', 'suffering', 'crime', 'noise', 'trouble', 'influence', 'space']

In [1014]:
def quantifiers(doc, unc_list): 
    docs = nlp(doc)
    erroneous = []
    uncount = ['much', 'less', 'least']
    uncount_of = ['deal', 'amount', 'smidge', 'much']
    count_pl = ['many', 'few', 'fewer', 'fewest', 'several', 'various', 'numerous', 'countless']
    count_pl_special = ['these', 'those'] 
    count_pl_of = ['number', 'numbers', 'couple', 'dozens', 'hundreds', 'thousands', 'millions', 'billions', 
                   'zillions', 'dozen', 'hundred','thousand', 'million', 'billion', 'zillion', 'one', 'each',
                   'every' , 'either', 'neither', 'both', 'several', 'few', 'various', 'countless', 'numerous', 
                   'another']
    count_sg_pl_quantified = ['each', 'every', 'another'] 
    uncount_count_sg = ['this', 'that', 'one']
 #needs uncountable words list to be checked    uncount_count_pl = ['enough'] #needs uncountable words list to be checked    uncount_count_pl_of = ['lot', 'lots', 'none', 'bags', 'heaps', 'loads', 'oodles', 'stacks'] 
    quantifiers = uncount + count_pl + uncount_count_sg + uncount_count_pl
    quantifiers_of = uncount_of + count_pl_of + uncount_count_pl_of
    for num, d in enumerate(docs.sents):
        for i in d:
            if i.text.lower() in quantifiers:
                if (i.text.lower() in uncount or i.text.lower() in uncount_count_sg)\
                and 'Plur' in i.head.morph.get("Number") and i.head.pos_ == 'NOUN':
                    if i.text.lower() == 'that' and (i.pos_ == 'SCONJ' or i.dep_ != 'det'):
                        pass
                    elif i.text.lower() == 'one' and (i.pos_ == 'PRON' or i.dep_ != 'nummod'):
                        pass
                    else:
                        erroneous.append([str(d), i.text.lower(), i.head, 'uncount_count_sg'])  
                elif i.text.lower() in count_pl and 'Sing' in i.head.morph.get("Number")\
                and i.head.pos_ == 'NOUN':
                    erroneous.append([str(d), i.text.lower(), i.head,'count_pl'])  
                elif i.text.lower() in uncount_count_pl and 'Sing' in i.head.morph.get("Number")\
                and i.head.text.lower() not in unc_list and 'NOUN' in i.head.pos_: 
                    erroneous.append([str(d), i.text.lower(), i.head,'enough'])
            elif i.text.lower() in quantifiers_of:
                for child in i.children:
                    if child.text.lower() == 'of':
                        for c in child.children:
                            if c.pos_ == 'NOUN':
                                if 'Sing' in c.morph.get("Number")[0] and i.text.lower() in count_pl_of:
                                    erroneous.append([str(d), i.text.lower(), c.text.lower(), 'count_pl_of']) 
                                elif 'Sing' in c.morph.get("Number")[0] and i.text.lower() in uncount_count_pl_of\
                                and c.text not in unc_list:
                                     erroneous.append([str(d), i.text.lower(), c.text.lower(), 'count_pl_of']) 
                                elif 'Plur' in c.morph.get("Number")[0] and i.text.lower() in uncount_of:
                                    erroneous.append([str(d), i.text.lower(), c.text.lower(),'uncount_of'])  
                            elif 'ADJ' in [str(x.pos_) for x in child.children]:
                                if i.text.lower() in uncount_of:
                                    erroneous.append([str(d), i.text.lower(), c.text.lower(),'uncount_of'])  
            elif i.text.lower() in count_sg_pl_quantified:
                if 'NUM' not in [str(x.pos_) for x in i.head.children]\
                and 'few' not in [str(x).lower() for x in i.head.children]:
                    if 'Plur' in i.head.morph.get("Number"):
                        erroneous.append([str(d), i.text.lower(), i.head.lower(),'count_sg_pl_quantified'])  
            elif i.text.lower() == 'either' or i.text.lower() == 'neither':
                if 'or' not in [str(x).lower() for x in i.head.children]\
                and 'nor' not in [str(x).lower() for x in i.head.children]:
                    if 'Plur' in i.head.morph.get("Number"):
                        erroneous.append([str(d), i.text.lower(), i.head.lower(),'either/neither'])  
            elif i.text.lower() == 'both':
                children_second_level = []
                if 'and' not in [str(x).lower() for x in i.head.children]\
                and 'ADP' not in [x.pos_ for x in i.children]:
                    for child in i.head.children:
                        children_second_level.append([str(x).lower() for x in child.children])
                    if 'and' in [item for sublist in children_second_level for item in sublist]:
                        pass
                    else:
                        if 'Sing' in i.head.morph.get("Number") and\
                        (i.head.pos_ == 'NOUN' or i.head.pos_ == 'PRON'):
                            if [str(d), i.text.lower(), i.head.lower(),'both'] not in erroneous:
                                erroneous.append([str(d), i.text.lower(), i.head.lower(),'both'])
            elif i.text.lower() in count_pl_special and i.dep_ == 'det':
                if i.head.text.lower() != 'kind' and i.head.text.lower() != 'type'\
                and i.head.text.lower() != 'sort':
                    if 'Sing' in i.head.morph.get("Number"):
                        erroneous.append([str(d), i.text.lower(), i.head.lower(),'count_pl_special'])
            elif i.pos_ == "NUM" and i.text.lower() != "one" and i.text != "1"\
            and (i.head.pos_ == "NOUN" or i.head.pos_ == "ADJ"):
                if re.match(r'hundreds|thousands|millions|billions|zillions', i.head.text.lower()):
                    erroneous.append([str(d), i.text.lower(), i.head, 'numerals'])
                
    return erroneous